# 第 3 章 字典和集合  
正是因为字典至关重要，Python 对它的实现做了高度优化，而散列表则是字典类型性能 出众的根本原因。  
集合(set)的实现其实也依赖于散列表。

## 3.1 泛映射类型

collections.abc 模块中有 Mapping 和 MutableMapping 这两个抽象基类，它们的作用是为 dict 和其他类似的类型定义形式接口。  
这些抽象基类的主要作用是作为形式化的文档，它们定义了构建一个映射类型所需要的最基本的接口。  
然后它们还可以跟 isinstance 一起被用来判定某个数据是不是广义上的映射类型：

In [5]:
from collections import abc

In [6]:
my_dict = {}
isinstance(my_dict, abc.Mapping)

True

如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现 __hash__() 方法。另外可散列对象还要有 __qe__() 方法，这样才能跟其他键做比较。如果两个可散列对象是相等的， 那么它们的散列值一定是一样的。

In [7]:
tt = (1, 2, (30, 40))
hash(tt)

-3907003130834322577

In [8]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [9]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

5149391500123939311

创建字典的不同方式

In [10]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})

a == b == c == d == e

True

## 3.2 字典推导  
字典推导(dictcomp)可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [13]:
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
]
country_code = {country : code for code, country in DIAL_CODES} # for 后面为内容，前面为 dict 的 key 和 value
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [14]:
{code : country.upper() for country, code in country_code.items() if code < 66}

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

## 3.3 常见的映射方法 —— 用setdefault处理找不到的键

In [39]:
"""创建一个从文件中找出单词到其出现位置的映射
返回值为 单词 [(行数, 单词首字母在行内的位置)]"""


import sys
import re

WORD_RE = re.compile(r'\w+')  # 将正则表达式的样式编译为一个正则表达式对象（正则对象），可以用于匹配
# \w = [A-Za-z0-9_]

index = {}
with open('file_for_exp3_2.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, start = 1):
        for match in WORD_RE.finditer(line):
            word = match.group()  # 将 Match类对象转化为 str 对象
            column_no = match.start()+1
            location = (line_no, column_no)
            # 这其实是一种很不好的实现，这样写只是为了证明论点 
            occurrences = index.get(word, [])  # 去 index 字典中获取 word 对应的位置的列表，如果找不到，则返回一个空列表
            occurrences.append(location)  # 将本次循环中找到的单词位置加入列表中
            index[word] = occurrences  # 将新获得的位置信息赋值给 index[word]
            
# 以字母顺序打印出结果    
for word in sorted(index, key=str.upper):
    print(word, index[word])

aaa [(1, 1), (1, 13)]
bbb [(1, 5)]
ccc [(1, 9)]
ddd [(2, 1)]
eee [(2, 5)]
fff [(2, 9)]
hello [(3, 1)]
hey [(3, 10)]
hi [(3, 7)]


In [40]:
import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}
with open('file_for_exp3_2.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no) 
            index.setdefault(word, []).append(location)
            
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper):
    print(word, index[word])

aaa [(1, 1), (1, 13)]
bbb [(1, 5)]
ccc [(1, 9)]
ddd [(2, 1)]
eee [(2, 5)]
fff [(2, 9)]
hello [(3, 1)]
hey [(3, 10)]
hi [(3, 7)]


setdefault 函数的作用：

In [43]:
my_dict = {}
key = 'abc'
new_value = 5

In [44]:
my_dict.setdefault(key, []).append(new_value)  

my_dict

{'abc': [5]}

等价于  

In [45]:
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value) 

my_dict

{'abc': [5, 5]}

note: enumerate(iterable, start=0)  
返回一个枚举对象。iterable 必须是一个序列，或 iterator，或其他支持迭代的对象。   
enumerate() 返回的迭代器的 \_\_next\_\_() 方法返回一个元组，里面包含一个计数值（从 start 开始，默认为 0）和通过迭代 iterable 获得的值。  

In [1]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']
list(enumerate(seasons))

[(0, 'Spring'), (1, 'Summer'), (2, 'Fall'), (3, 'Winter')]

In [2]:
list(enumerate(seasons, start=1))

[(1, 'Spring'), (2, 'Summer'), (3, 'Fall'), (4, 'Winter')]

note: re.finditer(pattern, string, flags=0)  
pattern 在 string 里所有的非重复匹配，返回为一个迭代器 iterator 保存了匹配对象。   
string 从左到右扫描，匹配按顺序排列。空匹配也包含在结果里。

In [27]:
import re

string = 'a123bc1cd22d23d'
pattern = '[a-z]([0-9]*)[a-z]'
for match in re.finditer(pattern, string):
    #print(type(match))
    s = match.start()
    e = match.end()
    print('String match "%s" at %d:%d' % (string[s:e], s, e))

String match "a123b" at 0:5
String match "c1c" at 5:8
String match "d22d" at 8:12


In [28]:
re_obj = re.compile(pattern)
for match in re_obj.finditer(string):
    s = match.start()
    e = match.end()
    print('String match "%s" at %d:%d' % (string[s:e], s, e))

String match "a123b" at 0:5
String match "c1c" at 5:8
String match "d22d" at 8:12


note: Match.group([group1, ...])  
返回一个或者多个匹配的子组。  
如果只有一个参数，结果就是一个字符串，如果有多个参数，结果就是一个元组（每个参数对应一个项），如果没有参数，组1默认到0（整个匹配都被返回）。

In [32]:
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist, Hello hhh")
m.group()

'Isaac Newton'

In [33]:
m.group(0)       # The entire match

'Isaac Newton'

In [34]:
m.group(1)       # The first parenthesized subgroup.

'Isaac'

In [35]:
m.group(2)       # The second parenthesized subgroup.

'Newton'

In [25]:
m.group(1, 2)    # Multiple arguments give us a tuple.

('Isaac', 'Newton')

In [31]:
for match in re_obj.finditer(string):
    word = match.group()
    print(word)
print("word class : ",type(word))
print("match class : ", type(match))

a123b
c1c
d22d
word class :  <class 'str'>
match class :  <class 're.Match'>
